In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# !pip install unet
# from google.colab import drive
# drive.mount('/content/drive')

import pandas as pd
import os

# path
dirr = '/content/drive/MyDrive/ColabNotebooks/lab2resources/sentence-completion'
# dataloader
questions = pd.read_csv(os.path.join(dirr,"testing_data.csv"))
answer_data = pd.read_csv(os.path.join(dirr,"test_answer.csv"))

import os,random,math,sys
import pandas as pd
import numpy as np
import nltk
import torch
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import ne_chunk,ne_chunk_sents,pos_tag,tree2conlltags
from nltk.corpus import words
import spacy
from spacy.lang.en import English
from spacy.tokenizer import Tokenizer
from tqdm import tqdm
nltk.download('words')
vocab=words.words()
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
# spacy_nlp = spacy.load("en_core_web_sm")

import csv
import sys, os
sys.path.append(os.getcwd()) # may need to change this
# from simple_classical_approach import *

class Question():
    def get_tokens(self,questions):
        return ["__START"]+tokenize(self.fields[questions.colnames["question"]])+["__END"]
    
    def getting_left_context(self,window=1,target="_____"):
        found=-1
        sent_tokens=self.get_tokens()
        for i,token in enumerate(sent_tokens):
            if token==target:
                found=i
                break  
            
        if found>-1:
            return sent_tokens[i-window:i]
        else:
            return []

    def choose_choices(self,lm,method="bigram",choices=[]):
        if choices==[]:
            choices=["a","b","c","d","e"]
        if method=="bigram":
            rc=self.get_right_context(window=1)
            lc=self.getting_left_context(window=1)
            probs=[lm.get_prob(rc[0],[self.getting_field(ch+")")],methodparams={"method":method.split("_")[0]})*lm.get_prob(self.getting_field(ch+")"),lc,methodparams={"method":method.split("_")[0]}) for ch in choices]
        elif method=="bigram_right":
            context=self.get_right_context(window=1)
            probs=[lm.get_prob(context[0],[self.getting_field(ch+")")],methodparams={"method":method.split("_")[0]}) for ch in choices]
        else:
            context=self.getting_left_context(window=1)
            probs=[lm.get_prob(self.getting_field(ch+")"),context,methodparams={"method":method.split("_")[0]}) for ch in choices]
        maxprob=max(probs)
        bestchoices=[ch for ch,prob in zip(choices,probs) if prob == maxprob]
        #if len(bestchoices)>1:
        #    print("Randomly choosing from {}".format(len(bestchoices)))
        return np.random.choice(bestchoices)
    
    def choosing_backoff(self,lm,methods=['bigram','unigram'],choices=["a","b","c","d","e"]):
        context=self.getting_left_context(window=1)
        probs=[lm.get_prob(self.getting_field(ch+")"),context,methodparams={"method":methods[0]}) for ch in choices]
        maxprob=max(probs)
        bestchoices=[ch for ch,prob in zip(choices,probs) if prob == maxprob]
        if len(bestchoices)>1:
            print("Backing off on {}".format(len(bestchoices)))
        return self.choose_choices(lm,choices=bestchoices,method=methods[1])
    
    def predictions(self,method="chooseA", model='mylm' ):
        if method=="chooseA":
            return self.chooseA()
        elif method=="random":
            return self.chooserandom()
        elif method=="bigram_backoff":
            return self.choosing_backoff(model,methods=["bigram","unigram"])
        else:
            return self.choose_choices(model,method=method)
    
class scc_reader:
    
    def __init__(self,qs=questions,ans=answer_data):
        self.qs=qs
        self.ans=ans
        self.reading_files()
        
    def reading_files(self):
        
        #read in the question file
        with open(self.qs) as instream:
            csvreader=csv.reader(instream)
            qlines=list(csvreader)
        
        #store the column names as a reverse index so they can be used to reference parts of the question
        questions.colnames={item:i for i,item in enumerate(qlines[0])}
        
        #create a question instance for each line of the file (other than heading line)
        questions=[questions(qline) for qline in qlines[1:]]
        
        #read in the answer file
        with open(self.ans) as instream:
            csvreader=csv.reader(instream)
            alines=list(csvreader)
            
        #add answer_data to question_data so predictions can be checked    
        for q,aline in zip(questions,alines[1:]):
            q.add_answer(aline)
        
    def getting_field(self,field):
        return [q.getting_field(field) for q in self.questions] 
    
    def predictions(self,method="chooseA"):
        return [q.predictions(method=method) for q in self.questions]
    
    def prediction_and_score(self,method="chooseA"):
        scores=[q.prediction_and_score(method=method) for q in questions]
        return sum(scores)/len(scores)

Question()
scc_reader

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


TypeError: ignored